In [6]:
# Niestandardowe pętle treningowe z własno zdefiniowanym Callbackiem

import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# --- Krok 1: Przygotowanie danych ---
# Wczytanie zbioru danych MNIST (obrazy cyfr 28x28 pikseli z etykietami od 0 do 9)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja wartości pikseli – przeskalowanie do zakresu 0–1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Tworzenie zbioru danych treningowych z podziałem na partie (batch size = 32)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# --- Krok 2: Definicja modelu ---
model = Sequential([
    Flatten(input_shape=(28, 28)),         # Spłaszczenie obrazu do wektora długości 784
    Dense(128, activation='relu'),         # Warstwa ukryta z 128 neuronami i aktywacją ReLU
    Dense(10)                               # Warstwa wyjściowa – po jednej wartości na każdą z 10 klas
])

# --- Krok 3: Funkcja straty, optymalizator, metryka ---
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Funkcja straty dla klasyfikacji wieloklasowej
optimizer = tf.keras.optimizers.Adam()  # Optymalizator Adam – skuteczny i popularny
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metryka dokładności

from tensorflow.keras.callbacks import Callback

# --- Krok 4: Własna callback-klasa ---
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Wypisuje wynik na końcu każdej epoki
        print(f'Koniec epoki {epoch + 1}, strata: {logs.get("loss")}, dokładność: {logs.get("accuracy")}')


# --- Krok 5: Niestandardowa pętla treningowa z użyciem własnego callbacka ---
epochs = 2
custom_callback = CustomCallback()  # Inicjalizacja własnego callbacka

for epoch in range(epochs):
    print(f'Start epoki {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Przejście w przód – model tworzy przewidywania
            logits = model(x_batch_train, training=True)
            # Obliczenie wartości straty
            loss_value = loss_fn(y_batch_train, logits)
        
        # Obliczenie gradientów względem wag
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Aktualizacja wag modelu
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Aktualizacja metryki dokładności
        accuracy_metric.update_state(y_batch_train, logits)

        # Co 200 kroków wypisujemy aktualny stan
        if step % 200 == 0:
            print(f'Epoka {epoch + 1} Krok {step}: Strata = {loss_value.numpy()} Dokładność = {accuracy_metric.result().numpy()}')
    
    # Na koniec epoki wywołujemy nasz własny callback
    custom_callback.on_epoch_end(epoch, logs={
        'loss': loss_value.numpy(),
        'accuracy': accuracy_metric.result().numpy()
    })

    # Resetowanie metryki przed kolejną epoką
    accuracy_metric.reset_state()  # Uwaga: reset_state() (nie reset_states)


Start epoki 1
Epoka 1 Krok 0: Strata = 2.317410707473755 Dokładność = 0.21875
Epoka 1 Krok 200: Strata = 0.42393308877944946 Dokładność = 0.8376865386962891
Epoka 1 Krok 400: Strata = 0.22326050698757172 Dokładność = 0.8682979941368103
Epoka 1 Krok 600: Strata = 0.1828051209449768 Dokładność = 0.8830074667930603
Epoka 1 Krok 800: Strata = 0.15207862854003906 Dokładność = 0.8954041600227356
Epoka 1 Krok 1000: Strata = 0.4218965470790863 Dokładność = 0.9026286005973816
Epoka 1 Krok 1200: Strata = 0.21066373586654663 Dokładność = 0.9094504714012146
Epoka 1 Krok 1400: Strata = 0.2375975102186203 Dokładność = 0.9144583940505981
Epoka 1 Krok 1600: Strata = 0.2724875807762146 Dokładność = 0.9179614186286926
Epoka 1 Krok 1800: Strata = 0.2214605212211609 Dokładność = 0.9220398664474487
Koniec epoki 1, strata: 0.0289167333394289, dokładność: 0.923966646194458
Start epoki 2
Epoka 2 Krok 0: Strata = 0.08052361011505127 Dokładność = 1.0
Epoka 2 Krok 200: Strata = 0.1263689547777176 Dokładność = 0.

In [ ]:
# Niestandardowe pętle treningowe z dodaną metryką dokładności:

import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# --- Krok 1: Przygotowanie danych ---
# Wczytanie zbioru danych MNIST (obrazy 28x28 pikseli przedstawiające cyfry 0-9)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja wartości pikseli (z zakresu 0-255 na zakres 0-1)
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Utworzenie zbioru treningowego z podziałem na partie (batch size = 32)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# --- Krok 2: Definicja modelu ---
# Model sekwencyjny: wejście → warstwa ukryta → wyjście
model = Sequential([ 
    Flatten(input_shape=(28, 28)),          # Spłaszczenie obrazu 28x28 pikseli do wektora 784 elementów
    Dense(128, activation='relu'),          # Warstwa ukryta z 128 neuronami i aktywacją ReLU
    Dense(10)                                # Warstwa wyjściowa z 10 neuronami (po jednym na każdą klasę 0-9)
])

# --- Krok 3: Definicja funkcji straty, optymalizatora i metryki ---
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Funkcja straty dla klasyfikacji wieloklasowej
optimizer = tf.keras.optimizers.Adam()  # Optymalizator Adam – szybka i skuteczna metoda aktualizacji wag
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metryka dokładności – ile etykiet zostało poprawnie przewidzianych


# --- Krok 4: Niestandardowa pętla treningowa z mierzoną dokładnością ---
epochs = 5  # Liczba epok (pełnych przejść przez dane treningowe)

for epoch in range(epochs):
    print(f'Start epoki {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Przejście w przód – model generuje przewidywania (logity)
            logits = model(x_batch_train, training=True)
            # Obliczenie straty porównując przewidywania z prawdziwymi etykietami
            loss_value = loss_fn(y_batch_train, logits)
        
        # Obliczenie gradientów względem wag modelu
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Zastosowanie gradientów – aktualizacja wag w modelu
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Zaktualizowanie metryki dokładności (dla aktualnej partii danych)
        accuracy_metric.update_state(y_batch_train, logits)

        # Wypisanie straty i dokładności co 200 kroków
        if step % 200 == 0:
            print(f'Epoka {epoch + 1} Krok {step}: Strata = {loss_value.numpy()} Dokładność = {accuracy_metric.result().numpy()}')
    
    # Reset metryki po zakończeniu każdej epoki, by zacząć mierzyć od zera
    accuracy_metric.reset_state()


In [ ]:
# Niestandardowe pętle treningowe:

import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np

# Wyłączenie ostrzeżeń Pythona
warnings.filterwarnings('ignore')

# Ustawienie poziomu logowania TensorFlow (0 = wszystkie logi, 2 = tylko błędy)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- Krok 1: Przygotowanie danych ---
# Załadowanie zestawu danych MNIST (obrazy cyfr 28x28 pikseli)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalizacja danych - przeskalowanie wartości pikseli z zakresu 0-255 do 0-1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Tworzenie obiektu zbioru treningowego z podziałem na partie (batch size 32)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# --- Krok 2: Zdefiniowanie modelu ---
# Model sekwencyjny z warstwami: spłaszczającą, ukrytą i wyjściową
model = Sequential([
    Flatten(input_shape=(28, 28)),         # Spłaszczenie obrazu 28x28 do wektora 784
    Dense(128, activation='relu'),         # Warstwa ukryta z 128 neuronami i aktywacją ReLU
    Dense(10)                               # Warstwa wyjściowa z 10 neuronami (po 1 dla każdej cyfry 0-9)
])

# --- Krok 3: Funkcja straty i optymalizator ---
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Funkcja straty dla klasyfikacji wieloklasowej
optimizer = tf.keras.optimizers.Adam()  # Optymalizator Adam

# --- Krok 4: Ręczna pętla treningowa (custom training loop) ---
epochs = 2  # Liczba epok (pełnych przejść przez dane)


# Pętla treningowa
for epoch in range(epochs):
    print(f'Start epoki {epoch + 1}')

    # Pętla po każdej partii danych
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Przejście w przód (forward pass)
            loss_value = loss_fn(y_batch_train, logits)   # Obliczenie wartości straty

        # Obliczenie gradientów i aktualizacja wag
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logowanie wartości straty co 200 kroków
        if step % 200 == 0:
            print(f'Epoka {epoch + 1}, krok {step}: strata = {loss_value.numpy()}')
